<a href="https://colab.research.google.com/github/whataLIN/DeepLearning/blob/main/whataLIN/ch09_DL_13_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
# Tab-delimited Bilingual Sentence Pairs
# 출처 : http://www.manythings.org/anki
!wget https://github.com/BigData23th/Data/raw/main/corpus.txt

--2023-04-07 02:06:27--  https://github.com/BigData23th/Data/raw/main/corpus.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BigData23th/Data/main/corpus.txt [following]
--2023-04-07 02:06:27--  https://raw.githubusercontent.com/BigData23th/Data/main/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253511 (248K) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>] 247.57K  --.-KB/s    in 0.003s  

2023-04-07 02:06:27 (73.0 MB/s) - ‘corpus.txt’ saved [253511/253511]



In [20]:
# 텍스트 파일을 전처리

import string # punctuation

l = [] # 특수문자를 지운 문장들을 받아줄 리스트

# with -> torch.no_grad()? => 특정한 객체가 생성이 되었을 때 with 구문이 끝나면 close 반환
with open("./corpus.txt", 'r', encoding='utf-8') as f:
    # open(경로, 'r', encoding=인코딩방식): 파일을 읽어와줌 (텍스트파일)
    # with .... -> 특정한 객체를 생성시키고, with 구문이 끝나면 해당 객체를 삭제 (반환)
    # open () as f -> open을 통해 읽어들여온 파일을 f라는 이름에 변수에 할당
    lines = f.read().split('\n') # '\n' = 엔터 = 개행문자
    # 파일을 읽어온 다음에, 엔터(줄) 기준으로 쪼개줘라 -> 문장별로 리스트화
    # lines = ['...', '...', '문장...']
    for line in lines: # 문장
        # 특수문자를 지우고 모든 글자를 소문자로 변경
        txt = "".join(v for v in line if not v in string.punctuation).lower()
        l.append(txt)

In [18]:
import string

l=[]

with open('./corpus.txt', 'r', encoding='utf-8') as f:  #with가 종료되면 open된 상태를 종료함
                                                    # 끝나면 f 를 erturn하겟음
  # print(f) - 불러온 파일 객체
  # print(f.read()) - 파일객체를 통으로 된 문자열로서 불러옴
  # testWrapper - 파이썬이 읽을 수 있는 형태가 되었음
  # print(f.read().split('\n'))   #모든 파일을 하나의 string으로 불러옴
  lines = f.read().split('\n')
  # for line in lines:
  #   txt = "".join(v for v in line if not v in string.punctuation).lower()
  #   l.append(txt)
  l+=["".join(v for v in line if not v in string.punctuation).lower()
      for line in f.read().split('\n')]


In [21]:
l[:5] #\t - 탭

['go\t가', 'hi\t안녕', 'run\t뛰어', 'run\t뛰어', 'who\t누구']

# 학습용 데이터 만들기

* 단어가 10개를 넘지 않는 문장들만 사용
* 문장을 불러올 때 <EOS(End Of Speech)> 토큰을 추가해서 문장이 끝났음을 알림

In [22]:
import numpy as np
import torch

from torch.utils.data.dataset import Dataset

## BOW를 만드는 함수 정의

In [24]:
def get_BOW(corpus): # 말뭉치 -> 문장 -> BOW를 만드는 함수
    BOW = {'<SOS>':0,'<EOS>':1}
    # BOW 안에 문장의 시작과 끝을 알리는
    # SOS(Start Of Speech) 토큰과 EOS(End Of Speech) 토큰을 추가

    # 문장 내 단어들을 사용하여 BOW를 생성
    for line in corpus:
        for word in line.split():
            if word not in BOW.keys(): # 등록되지 않은 단어면
                BOW[word] = len(BOW.keys())
                # 사전에 추가해주는데, 해당 단어의 고유번호는 이전까지의 키의 갯수
    return BOW

## 학습용 데이터셋 정의

In [41]:
class Eng2Kor(Dataset):
    def __init__(self, path='./corpus.txt') -> None:
        super().__init__()
        self.eng_corpus = [] # 영어문장이 들어가는 변수
        self.kor_corpus = [] # 한글문장이 들어가는 변수

        with open(path, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')
            for line in lines: # 문장
                txt = "".join(v for v in line
                              if not v in string.punctuation).lower()
                # \t 구분이 되어 있었음 (영어와 한글) -> 탭을 기준으로 분리
                engtxt, kortxt = txt.split('\t') # 0 : 영어 # 1 : 한글
                # engtxt = txt.split('\t')[0]
                # kortxt = txt.split('\t')[1]

                # 길이가 10 이하인 문장 = 단어의 갯수가 10개 이하인 문장만 학습
                if not (len(engtxt.split()) <= 10 and len(kortxt.split())<=10):
                    continue
                    # 영어, 한글 번역문 모두 10개 단어 이하인 데이터만 사용
                self.eng_corpus.append(engtxt)
                self.kor_corpus.append(kortxt)
        
        # 영어와 한글 문장을 각각 BOW(단어 사전)으로 변환
        self.engBOW = get_BOW(self.eng_corpus)
        self.korBOW = get_BOW(self.kor_corpus)
    
    # 문장을 단어별로 분리하는 함수
    def gen_seq(self, line): # line = 문장
        seq = line.split()  # 토큰화 한다음에
        seq.append("<EOS>")  # 마지막에 EOS(문장 끝) 토큰 추가
        return seq

    def __len__(self): # 데이터의 개수를 반환하는 함수
        return len(self.eng_corpus)

    # 데이터와 정답을 반환하는 함수
    def __getitem__(self, i): # data, label을 지정
        # 문자열로 되어 있는 문장을 숫자 표현으로 변경
        # 1) 영어 corpus 중 i번째 문장을 받아옴
        # 2) gen_seq -> i번째 문장을 seq 형태로 변환 (토큰+EOS)
        # 3) 단어 사전을 사용해서 고유번호 형태로 변환 (학습을 위해 숫자형태로 변환)
        #self.eng_corpus[i] -> self.gen_seq(self.eng_corpus[i])
        # -> 리스트컴프리헨션으로 대칭시킹 : [ for in self.gen_seq(self.eng_corpus)]
        data = np.array([
            [self.engBOW[txt] for txt in self.gen_seq(self.eng_corpus[i])]
        ])
        label = np.array([
            [self.korBOW[txt] for txt in self.gen_seq(self.kor_corpus[i])]
        ])
        return data, label # 영어 데이터 (입력) -> 한글 데이터 (정답)

## 데이터 로더

In [42]:
def loader(dataset): # 데이터셋의 문장을 한 문장씩 불러오기 위한 함수 정의
    for i in range(len(dataset)):
        data, label = dataset[i]

        # 데이터와 정답을 반환
        yield torch.tensor(data), torch.tensor(label)
        # yield : 리턴과 유사, 값을 반복적으로 반환

# 모델 정의

## 인코더 정의
* 임베딩층, GRU층

In [43]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size) -> None:
        super(Encoder, self).__init__()

        # 임베딩층
        self.embedding = nn.Embedding(input_size, hidden_size)   #-과소표현으로

        # GRU층
        self.gru = nn.GRU(hidden_size, hidden_size)     #디폴트1로
        # nn.GRU : GRU 계산. input_size, hidden_size, num_layers)
    
    def forward(self, x, h): # x: 입력값 / h : 은닉상태
        # 배치 차원과 시계열 차원 추가
        x = self.embedding(x).view(1,1,-1) #1,1이 배치/시계열차원
        output, hidden = self.gru(x,h)    # output : 문장의 특성, hidden 은닉 상태
        return output, hidden

## 디코더 정의
* 임베딩 층
* 전결합 층 (ReLU)
* 전결합 층 (Softmax)
* 내적
* GRU층

In [44]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=11) -> None:
        super(Decoder, self).__init__()

        # 임베딩 층 정의
        self.embedding = nn.Embedding(output_size, hidden_size)

        # 어텐션 가중치를 계산하기 위한 MLP층
        self.attention = nn.Linear(hidden_size * 2, max_length)

        # 10개 + <EOS>(1) = 최대 길이 11개


        # 특징 추출을 위한 MLP층
        self.context = nn.Linear(hidden_size *2, hidden_size)

        # 오버피팅을 피하기 위한 드롭아웃층
        self.dropout = nn.Dropout(dropout_p)    #외부에서 수정 가능하게.

        # GRU층
        self.gru = nn.GRU(hidden_size, hidden_size) #연속적 의미 분류

        # 단어 분류를 위한 MLP층
        self.out = nn.Linear(hidden_size, output_size)

        # 활성화 함수
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

        # LogSoftmax(dim) : 소프트맥스 함수에 로그 값을 취한 것을 반환
        # dim -> 계산의 대상이 될 차원값
    
    def forward(self, x, h, encoder_outputs): # x : 입력값, h : 은닉상태, e...: 인코더 결과값
        # 입력 받은 x(현 시점의 디코더 입력)을 임베딩 층을 사용해 밀집 표현으로 변환
        
        x = self.embedding(x).view(1,1,-1)      # 배치 차원, 시계열 차원, 단어들의 개수.
        x = self.dropout(torch.xlogy_)        

        # 어텐션 가중치 계산
        attn_weights = self.softmax(
            self.attention(torch.cat((x[0], h[0], -1)))   #배치차원의 0번째, 은닉츠으이 0번째를 가지고 처리
        )

        # 어텐션 가중치와 인코더의 출력을 내적(크기가 다른 두 배열을 방향이 일치하는 만큼 곱함)
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)   #가장 밖 차원을 unsqueeze
        ) # bmm(A, B) : A 크기가 (B, N, M)이고, B 크기가 (B, M, K)
        # => (B, N, K) 크기의 출력을 반환 > 유사도 구하기 (어텐션가중치-이전까지의 전체맥락 - 인코더출력(일치도 얼마?))

        # 인코더 각 시점의 중요도와 밀집 표현을 합쳐서 MLP층으로 특징 추출
        output = torch.cat((x[0], attn_applied[0]), 1)    #각 시점 중요도가 x의0
        output = self.context(output).unsqueeze(0)
        output = self.relu(output)
        # 인코더의 중요도(attn_applied)와 현시점에서의 디코더의 밀집표현(x)을 합쳐서
        # MLP층(context)으로 입력
        # -> MP층은 인코더 각 시점의 중요도와 현시점 디코더의 밀집표현을 동시에 처리
        # -> 인코더의 중요도가 디코더의 반영

        # GRU층으로 입력
        output, hidden = self.gru(output, hidden)

        # 예측된 단어를 출력
        output = self.out(output[0])

        return output

# 학습 정의

## 학습에 필요한 요소 정의

In [45]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam

# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else 'cpu'
# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더 디코더 정의
encoder = Encoder(input_size = len(dataset.engBOW), hidden_size = 64).to(device)  #던져줌
decoder = Decoder(64, len(dataset.korBOW), dropout_p = 0.1).to(device)

# 인코더와 디코더 학습을 위한 최적화 함수 정의
encoder_optimizer = Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = Adam(decoder.parameters(), lr=0.001)

In [34]:
device

'cuda'

## 학습 루프 정의

In [ ]:
for epoch in range(200):
    iterator = tqdm(loader(dataset), total=len(dataset))
    total_loss = 0

    for data, label in iterator:
        data = torch.tensor(data, dtype=torch.long).to(device)
        label = torch.tensor(label, dtype=torch.long).to(device)

        # 인코더의 초기 은닉 상태
        encoder_hidden = torch.zeros(1, 1, 64).to(device)    #특징이 64개로
        # 인코더의 모든 시점의 출력을 저장하는 변수
        # 최대 단어 10개 + 종료(EOS) -> 11개
        encoder_outputs = torch.zeros(11,64).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        loss = 0

        # 인코더 동작
        for ei in range(len(data)): # data : 토큰화, 고유번호 -> 단어들의 리스트
            # ei => data의 인덱스들
            # 한 단어씩 인코더에 넣어줌
            encoder_output, encoder_hidden = encoder(data[ei], encoder_hidden)

            # 인코더의 은닉상태를 저장
            encoder_output[ei] = encoder_output[0,0]    #64를 불러냄
        
        decoder_input = torch.tensor([[0]]).to(device)


        # 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로 지정
        decoder_hidden = encoder_hidden

        # 디코더 동작
        # 티처 포싱 (Teacher Forcing: 교사 강요)
        # Seq2Seq 구조에서 현시점의 입력을 (모델의 예측값을 사용하는 대신에) 정답을 이용하는 방법
        # 엉뚱한 답을 피하고, 시간 단축을 위해 강제적으로 정답을 넣어주는 기술 (50% 확률로 적용)
        use_teacher_forcing = True if random.random() < 0.5 else False
        
        if use_teacher_forcing:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )


                # 직접적으로 정답을 다음 시점의 입력으로 넣어줌
                target = torch.tensor(label[di], dtype = torch.long).to(device) 
                target = torch.unsqueeze(target, dim=0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                decoder_input = target #바꿔치기

                #데이터의 인덱스로 라벨값 호출 - 

        else:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                
                target = torch.tensor(label[di], dtype = torch.long).to(device) 
                target = torch.unsqueeze(target, dim=0).to(device)

                # 가장 높은 확률을 갖는 단어의 인덱스 topi
                topv, topi = decoder_output.topk(1)   # top k -> 가장 높은 확률을 가진 (1)개를 불러옴
                # 디코더의 예측값을 다음 시점의 입력으로 넣어줌
              
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                
                if decoder_input.item() == 1: #<EOS> 토큰을 만나면 중지
                    break
        
        # 전체 손실 계산
        total_loss += loss.item() / len(dataset)
        iterator.set_description(f"epoch:{epoch+1} loss:{total_loss}")
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

torch.save(encoder.state_dict(), "attn_enc.pt")
torch.save(decoder.state_dict(), "attn_dec.pt")

In [46]:
for epoch in range(200):
    iterator = tqdm(loader(dataset), total=len(dataset))
    total_loss = 0

    for data, label in iterator:
        data = torch.tensor(data, dtype=torch.long).to(device)
        label = torch.tensor(label, dtype=torch.long).to(device)

        # 인코더의 초기 은닉 상태
        encoder_hidden = torch.zeros(1, 1, 64).to(device)
        # 인코더의 모든 시점의 출력을 저장하는 변수
        # 최대 단어 10개 + 종료(EOS) -> 11개
        encoder_outputs = torch.zeros(11, 64).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        loss = 0

        # 인코더 동작
        for ei in range(len(data)): # data : 토큰화, 고유번호 -> 단어들의 리스트
            # ei => data의 인덱스들
            # 한 단어씩 인코더에 넣어줌
            encoder_output, encoder_hidden = encoder(
                data[ei], encoder_hidden)

            # 인코더의 은닉상태를 저장
            encoder_outputs[ei] = encoder_output[0, 0]
            # 1, 1, *64*
        
        decoder_input = torch.tensor([[0]]).to(device)

        # 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로 지정
        decoder_hidden = encoder_hidden

        # 디코더 동작
        # 티처 포싱 (Teacher Forcing: 교사 강요)
        # Seq2Seq 구조에서 현시점의 입력을 (모델의 예측값을 사용하는 대신에) 정답을 이용하는 방법
        # 엉뚱한 답을 피하고, 시간 단축을 위해 강제적으로 정답을 넣어주는 기술 (50% 확률로 적용)
        use_teacher_forcing = True if random.random() < 0.5 else False
        
        if use_teacher_forcing:
            for di in range(len(label)): # di : data 인덱스
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )

                target = torch.tensor(label[di], dtype=torch.long).to(device)                
                target = torch.unsqueeze(target, dim=0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)
                
                # 직접적으로 정답을 다음 시점의 입력으로 넣어줌
                decoder_input = target # 바꿔치기
        else:
            for di in range(len(label)):
                decoder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

                target = torch.tensor(label[di], dtype=torch.long).to(device)                
                target = torch.unsqueeze(target, dim=0).to(device)
                loss += nn.CrossEntropyLoss()(decoder_output, target)

                # 가장 높은 확률을 갖는 단어의 인덱스 topi
                topv, topi = decoder_output.topk(1) # top k -> (1)개를 불러옴
                
                # 디코더의 예측값을 다음 시점의 입력으로 넣어줌
                decoder_input = topi.squeeze().detach() # 텐서 -> 값
                
                if decoder_input.item() == 1: #<EOS> 토큰을 만나면 중지
                    break
        
        # 전체 손실 계산
        total_loss += loss.item() / len(dataset)
        iterator.set_description(f"epoch:{epoch+1} loss:{total_loss}")
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

torch.save(encoder.state_dict(), "attn_enc.pt")
torch.save(decoder.state_dict(), "attn_dec.pt")

  0%|          | 0/3592 [00:00<?, ?it/s]

<ipython-input-46-b67003997b3d>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.long).to(device)
<ipython-input-46-b67003997b3d>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long).to(device)


RuntimeError: ignored

In [47]:
!wget https://github.com/BigData23th/Data/raw/main/attn_enc.pt
!wget https://github.com/BigData23th/Data/raw/main/attn_dec.pt

--2023-04-07 03:42:12--  https://github.com/BigData23th/Data/raw/main/attn_enc.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BigData23th/Data/main/attn_enc.pt [following]
--2023-04-07 03:42:12--  https://raw.githubusercontent.com/BigData23th/Data/main/attn_enc.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727147 (710K) [application/octet-stream]
Saving to: ‘attn_enc.pt.1’

attn_enc.pt.1       100%[===================>] 710.10K  --.-KB/s    in 0.007s  

2023-04-07 03:42:12 (103 MB/s) - ‘attn_enc.pt.1’ saved [727147/727147]

--2023-04-07 03:42:13--  https://github.com/BigData23th/Dat

# 성능 평가

In [51]:
# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

<All keys matched successfully>

In [52]:
# 불러올 영어 문장을 랜덤하게 지정
idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
input_sentence

'tom is a timid child'

In [54]:
# 신경망이 번역한 문장
pred_sentence = ""

In [55]:
data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device) # 영어문장
label = torch.tensor(label, dtype=torch.long).to(device) # 한국어문장

In [56]:
data

tensor([[ 52, 198, 271, 751, 900,   1]], device='cuda:0')

In [57]:
label

tensor([[ 103, 1153, 1450,    1]], device='cuda:0')

## 인코더 동작

In [58]:
# 인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기 위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

In [59]:
for ei in range(len(data)):
    # 한 단어씩 인코더에 넣어줌
    encoder_output, encoder_hidden = encoder(
        data[ei], encoder_hidden
    )
    # 인코더의 출력을 저장
    encoder_outputs[ei] = encoder_output[0, 0]

RuntimeError: ignored

In [ ]:
encoder_outputs

## 디코더 동작

In [ ]:
# 디코더 초기 입력
decoder_input = torch.tensor([[0]]).to(device)
# 0 -> 문장이 시작되었다는 SOS 토큰

# 인코더의 마지막 은닉 상태 -> 디코더의 초기 은닉 상태
decoder_hidden = encoder_hidden

In [ ]:
for di in range(11):
    # 디코더 모델을 통해서 단어별 나올 확률
    decoder_output = decoder(
        decoder_input, decoder_hidden, encoder_outputs
    )
    # 가장 높은 확률을 갖는 단어의 요소 계산
    topv, topi = decoder_output.topk(1)
    # 가장 높은 확률의 단어
    decoder_input = topi.squeeze().detach()

    # EOS 토큰을 만나면 중지
    if decoder_input.item() == 1:
        break
    
    # 예측 문자열에 가장 높은 확률의 단어를 추가
    pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "

print(input_sentence)
print(pred_sentence)

i just want to rest
싶어 싶어 싶어 싶어 이걸 어제 이걸 싶어 쉬고 싶어 이걸 함께 함께 이걸 싶어 싶어 싶어 싶어 싶어 싶어 싶어 이걸 


## 통합

In [ ]:
!pip install gtts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.0 MB/s eta 0:00:00


In [ ]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam

# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else 'cpu'
# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더 디코더 정의
encoder = Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p=0.1).to(device)

# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
# 디코더 가중치 불러오기
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
# 신경망이 번역한 문장
pred_sentence = ""

data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device)
label = torch.tensor(label, dtype=torch.long).to(device)

encoder_hidden = torch.zeros(1, 1, 64).to(device)
encoder_outputs = torch.zeros(11, 64).to(device)

for ei in range(len(data)):
   encoder_output, encoder_hidden = encoder(
       data[ei], encoder_hidden)
     
   encoder_outputs[ei] = encoder_output[0, 0]  

decoder_input = torch.tensor([[0]]).to(device)

decoder_hidden = encoder_hidden 

for di in range(11):
   decoder_output = decoder(
                       decoder_input, decoder_hidden, encoder_outputs)
   topv, topi = decoder_output.topk(1)
   decoder_input = topi.squeeze().detach()

   if decoder_input.item() == 1:  
       break

   pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "  

from gtts import gTTS
from IPython.display import Audio
from time import sleep

file_name = '/content/sample.mp3'

text = input_sentence
tts_en = gTTS(text=text)
tts_en.save(file_name)

print(input_sentence)  # 영어 문장
wn = Audio(file_name, autoplay=True)
display(wn)

sleep(4)

text = pred_sentence
tts_ko = gTTS(text=text, lang='ko')
tts_ko.save(file_name)
print(pred_sentence)  # 한글 문장

wn = Audio(file_name, autoplay=True)
display(wn)

any questions


아무 질문이라도 
